In [4]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import pprint 
import matplotlib.pyplot as pl
import time
import datetime
import keys as k

Matplotlib is building the font cache; this may take a moment.


In [5]:
key = k.credentials['DEVELOPER_KEY']

In [6]:
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [11]:
def youtube_search(q, max_results=50,order="relevance",nextPage_token = None, location=None, location_radius=None):

    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=key)

    search_response = youtube.search().list(
    q=q,
    type="video",
    order = order,
    part="id,snippet", # Part signifies the different types of data you want 
    maxResults=max_results,
    pageToken = nextPage_token, 
    location=location,
    locationRadius=location_radius,
    videoDuration = 'medium',
    publishedAfter = '2020-01-01T00:00:00Z').execute()
    
    nextPage_token = search_response.get("NextPageToken")



#   the attributes that we want to be able to view in the dataset
    title = []
    channelId = []
    channelTitle = []
    categoryId = []
    videoId = []
    viewCount = []
    likeCount = []
    dislikeCount = []
    commentCount = []
    favoriteCount = []
    category = []
    tags = []
    videos = []
    thumbnail_url = []
    publishTime = []
    description = []
    
    for search_result in search_response.get("items", []):
        #pprint.pprint(search_result)
  
  
        if search_result["id"]["kind"] == "youtube#video":

            title.append(search_result['snippet']['title']) 

            videoId.append(search_result['id']['videoId'])

            response = youtube.videos().list(
                part='statistics, snippet',
                id=search_result['id']['videoId']).execute()

            channelId.append(response['items'][0]['snippet']['channelId'])
            channelTitle.append(response['items'][0]['snippet']['channelTitle'])
            categoryId.append(response['items'][0]['snippet']['categoryId'])
            favoriteCount.append(response['items'][0]['statistics']['favoriteCount'])
            viewCount.append(response['items'][0]['statistics']['viewCount'])
            thumbnail_url.append(response['items'][0]['snippet']['thumbnails']['default']['url'])
            publishTime.append(response['items'][0]['snippet']['publishedAt'])
            description.append(response['items'][0]['snippet']['description'])

 
        if 'commentCount' in response['items'][0]['statistics'].keys():
            commentCount.append(response['items'][0]['statistics']['commentCount'])
        else:
            commentCount.append([])
        if 'likeCount' in response['items'][0]['statistics'].keys():
            likeCount.append(response['items'][0]['statistics']['likeCount'])
        else:
            likeCount.append([])
        if 'dislikeCount' in response['items'][0]['statistics'].keys():
            dislikeCount.append(response['items'][0]['statistics']['dislikeCount'])
        else:
            dislikeCount.append([])
	  
        if 'tags' in response['items'][0]['snippet'].keys():
            tags.append(response['items'][0]['snippet']['tags'])
        else:
            tags.append([])
#     pprint.pprint(response)
    youtube_dict = {'tags':tags,'channelId': channelId,'channelTitle': channelTitle,'categoryId':categoryId,'title':title,'videoId':videoId,'viewCount':viewCount,'likeCount':likeCount,'dislikeCount':dislikeCount,'commentCount':commentCount,'favoriteCount':favoriteCount, 'publishTime':publishTime, 'thumbnail_url': thumbnail_url, 'description': description}

    return youtube_dict

In [12]:
endsars = youtube_search('endsars')

In [13]:
endsars = pd.DataFrame(endsars)

In [14]:
endsars.head()

,tags,channelId,channelTitle,categoryId,title,videoId,viewCount,likeCount,dislikeCount,commentCount,favoriteCount,publishTime,thumbnail_url,description
0,"[IGP, Police, SARS, ENDSARS, Endsars, #endsars...",UCEXGDNclvmg6RW0vipJYsTQ,Channels Television,25,Security Operatives Open Fire At Lekki #EndSAR...,BwP_-UU72aw,149941,[],[],328,0,2020-10-20T19:14:45Z,https://i.ytimg.com/vi/BwP_-UU72aw/default.jpg,Don't forget to subscribe: https://bit.ly/2Hb...
1,"[Channels Television, Channels TV, Africa News...",UCEXGDNclvmg6RW0vipJYsTQ,Channels Television,25,#ENDSARS Protest: Hoodlums Storm Correctional ...,oRaptDy_2HI,736003,[],[],874,0,2020-10-19T11:23:34Z,https://i.ytimg.com/vi/oRaptDy_2HI/default.jpg,Don't forget to subscribe: https://bit.ly/2Hb...
2,"[Channels Television, Channels TV, Africa News...",UCEXGDNclvmg6RW0vipJYsTQ,Channels Television,25,#EndSARS Protest In Edo Turns Bloody As Thugs ...,HtuMsNiubKM,1185026,[],[],1060,0,2020-10-16T17:33:56Z,https://i.ytimg.com/vi/HtuMsNiubKM/default.jpg,Don't forget to subscribe: https://bit.ly/2Hb...
3,"[Policy, Politics, Pop Culture, News, Nigeria,...",UC--v8aRh2BQDgfMEAz1t9tw,TheSheet TV,24,"#EndSARS, Abuja Protest Police Use Gas and Fi...",vO6BnvkVVfs,27099,119,7,10,0,2020-10-09T12:25:41Z,https://i.ytimg.com/vi/vO6BnvkVVfs/default.jpg,"The News Platform for Policy, Pop Culture & Po..."
4,"[Channels Television, Channels TV, Africa News...",UCEXGDNclvmg6RW0vipJYsTQ,Channels Television,25,ENDSARS: Police Will Never Again Accept Such V...,_-yJe_X1yPw,16646,[],[],147,0,2020-12-09T12:19:23Z,https://i.ytimg.com/vi/_-yJe_X1yPw/default.jpg,Don't forget to subscribe: https://bit.ly/2Hb...


In [23]:
import time

def save_to_csv(endsars):
    path = os.getcwd() + '\data\\'
    current_timestamp = time.strftime("%y%m%d_%H%M%S")

    if not os.path.exists(path):
        os.mkdir(path)

    file_name = current_timestamp+"_youtube_data.csv"
    
    full_path = os.path.join(path, file_name)
    
    endsars.to_csv(full_path, index=False)
    
    return full_path


In [24]:
path = save_to_csv(endsars)
print("File created successfully. File save to: {0}".format(path))

File created successfully. File save to: C:\Users\ANDRE\PROJECTS\youtube-data-extraction\data\201220_182249_youtube_data.csv
